In [ ]:
from tespy.connections import HumidAirConnection, Connection, Ref
from tespy.components import Source, Sink, MovingBoundaryHeatExchanger
from tespy.networks import Network

from CoolProp.CoolProp import HAPropsSI

In [ ]:
nw = Network()
nw.units.set_defaults(
    temperature="°C",
    pressure="bar",
    heat="kW"
)

In [ ]:
def get_water_air_mixture_fractions_pTR(p, T, R):
    w = HAPropsSI("W", "P", p, "T", T, "R", R)
    return get_water_air_mixture_fractions_w(w)


def get_water_air_mixture_fractions_w(w):
    air = 1 / (1 + w)
    return {"air": air, "water": 1 - air}

In [ ]:
from tespy.tools.fluid_properties.functions import _get_humid_air_humidity_ratio


def _calculate_dQ_freezing_water(T_evap, T_air, air_fluid_data, air_pressure, air_mass_flow):
    dh_water = 4000 * (T_air - 273.15)
    dh_freeze = 333000
    dh_ice = 2000 * (273.15 - T_evap)
    w_actual = _get_humid_air_humidity_ratio(air_fluid_data)
    w_saturated = HAPropsSI("W", "P", air_pressure, "T", T_air, "R", 1)

    m_water_freeze = (w_actual - w_saturated) * air_mass_flow
    return (dh_water + dh_freeze + dh_ice) * m_water_freeze


class MovingBoundaryHeatExchangerWithFreeze(MovingBoundaryHeatExchanger):

    def energy_balance_func(self):
        residual = super().energy_balance_func()
        T_evap = self.outl[1].calc_T()
        if T_evap < 273.15:
            T_air_out = self.outl[0].calc_T()
            if T_air_out > 273.15:
                dQ_freeze = _calculate_dQ_freezing_water(
                    T_evap, T_air_out, self.outl[0].fluid_data, self.outl[0].p.val_SI, self.inl[0].m.val_SI
                )
                residual -= dQ_freeze

        return residual

    def energy_balance_dependents(self):
        return [
            self.inl[0].m, self.inl[0].h,
            self.outl[0].p, self.outl[0].h,
            self.inl[1].m, self.inl[1].h,
            self.outl[1].p, self.outl[1].p
        ]

    def calc_parameters(self):
        super().calc_parameters()
        self.Q.val_SI = -self.inl[1].m.val_SI * (self.outl[1].h.val_SI - self.inl[1].h.val_SI)

In [ ]:
so = Source("source")
hex = MovingBoundaryHeatExchangerWithFreeze("heat exchanger")
si = Sink("sink")

so2 = Source("refrigerant source")
si2 = Sink("refrigerant sink")

a1 = Connection(so2, "out1", hex, "in2", label="a1")
a2 = Connection(hex, "out2", si2, "in1", label="a2")

c1 = Connection(so, "out1", hex, "in1", label="c1")
c2 = Connection(hex, "out1", si, "in1", label="c2")

nw.add_conns(a1, a2, c1, c2)

a1.set_attr(fluid={"NH3": 1}, m=2, x=0.3)
a2.set_attr(td_dew=5, T_dew=-20)

c1.set_attr(
    fluid0={"water": 0.0005, "air": 0.9995},#get_water_air_mixture_fractions_pTR(p=1e5, T=268.16, R=1),
    p=1,
    T=15,
    mixing_rule="humidair",
    r=1,
    fluid_balance=True,
    h0=0
)
c2.set_attr(T=5, h0=0)

hex.set_attr(dp1=0, dp2=0)

nw.solve("design")

In [ ]:
nw.print_results()

In [ ]:
heat, T_hot, T_cold, _, _ = hex.calc_sections()

In [ ]:
from matplotlib import pyplot as plt


plt.plot(heat, T_hot)
plt.plot(heat, T_cold)